In [1]:
import os
from torch.utils.data import Dataset, DataLoader, ConcatDataset,  WeightedRandomSampler
from skimage import io, transform
from torchvision import transforms, utils
import torch
import numpy as np
import nibabel as nib
from random import randint
from PIL import Image
import torch.optim as optim
import time
import QuickNAT as QN
import torch.nn as nn
import matplotlib.pyplot as plt

### The dataset of T1w image and brainmask image in axial plane

In [2]:
class TrainDataset(Dataset):
    """Training dataset with mask image mapping to classes"""
    def __init__(self, T1a_dir, brainmask_dir, transform=None):
        """
        Args:
            T1a_dir (string): Directory with T1w image in axial plane
            transform (callable): Optional transform to be applied on a sample
            brainmask_dir (string): Directory with brainmask in axial plane
        """
        self.T1a_dir = T1a_dir
        self.transform = transform
        self.brainmask_dir = brainmask_dir
#         self.mapping = {
#             180:91
#         }
        
#     def mask_to_class(self, mask):
#         for k in self.mapping:
#             mask[mask==k] = self.mapping[k]
#         return mask
    
    def __len__(self):
        T1a_list = os.listdir(self.T1a_dir)
        return len(T1a_list)
    
    
    def __getitem__(self, idx):
        T1a_list = os.listdir(self.T1a_dir)
        brainmask_list = os.listdir(self.brainmask_dir)
        
        T1a_str = T1a_list[idx]
        
        T1a_arr = io.imread(os.path.join(self.T1a_dir, T1a_str))
        T1a_tensor = torch.from_numpy(T1a_arr)
        
        compose_T1 = transforms.Compose([transforms.ToPILImage(),
                                         transforms.Resize((128,128),interpolation=Image.NEAREST),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        T1a_tensor = torch.unsqueeze(T1a_tensor, dim = 0)   

        T1a_tensor = compose_T1(T1a_tensor)
        
        # The original brainmask value is 0,1,2     
        brainmask_str = brainmask_list[idx]
    
        brainmask_arr = io.imread(os.path.join(self.brainmask_dir, brainmask_str))
        brainmask_tensor = torch.from_numpy(brainmask_arr)
        
        compose = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Resize((128,128),interpolation=Image.NEAREST), 
                                      transforms.ToTensor()])
        
        brainmask_tensor = torch.unsqueeze(brainmask_tensor, dim = 0)
        brainmask_tensor = compose(brainmask_tensor)
        brainmask_tensor = brainmask_tensor.squeeze()
        
        brainmask_tensor = torch.round(brainmask_tensor / 0.0039).byte()
        
#         parc1a_tensor = self.mask_to_class(parc1a_tensor)
      
        sample = {'T1a':T1a_tensor, 'brainmask':brainmask_tensor}
        
        if self.transform:
            T1a = self.transform(T1a_tensor)
            sample = {'T1a':T1a, 'brainmask':brainmask}
            
        return sample

In [3]:
sub_idx = 0
slice_idx = 1
T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
brainmask_dir = '/home/xiaoyu/MRIdata_group/brainmask/axial/sub{}/group{}'.format(sub_idx,slice_idx)
total_data = TrainDataset(T1a_dir=T1a_dir, brainmask_dir = brainmask_dir)

#### There are only 3 classes in brainmask

In [4]:
sample = total_data[0]
target = sample['brainmask']
image = sample['T1a']

In [5]:
for sub_idx in range(1,330):
    T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
    brainmask_dir = '/home/xiaoyu/MRIdata_group/brianmask/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
    train_data = TrainDataset(T1a_dir=T1a_dir, brainmask_dir = brainmask_dir)
    total_data = total_data + train_data
print(len(total_data))

3300


In [7]:
colors = torch.tensor([])
for i in range(len(total_data)):
    print('\n')
    sample = total_data[i]
    target = sample['brainmask']
    unique_color, count_ind = np.unique(target, return_counts = True)
    print(unique_color)
    print(count_ind)
    colors = torch.cat((colors,torch.tensor(unique_color).float()))
    unique_color, count_conc = np.unique(colors, return_counts = True)
    print('------')
    print(unique_color)
    print(count_conc)
    if i ==1:
        break
    



[0 1 2]
[15931   441    12]
------
[0. 1. 2.]
[1 1 1]


[0 1 2]
[15971   401    12]
------
[0. 1. 2.]
[2 2 2]


In [8]:
sample = train_data[0]
target = sample['brainmask']
print(target.size())

FileNotFoundError: [Errno 2] No such file or directory: '/home/xiaoyu/MRIdata_group/brianmask/axial/sub329/slice1'

In [33]:
unique_color, count = np.unique(target, return_counts = True)
print(unique_color)
print(count.size)
weight = 1. / count
        
sample_weight = torch.tensor([])
        
for x in range(target.size(0)):
    for y in range(target.size(1)):
        sample_weight = weight[target[x,y]]

[  0   1   2   3   4   5   6   7   8   9  10  11  13  14  15  16  17  18
  19  21  30  31 101 106 112 113 117 118 119 120 123 124 125 126 127 128
 129 130 131 132 133 135 137 138 139 140 141 142 143 146 154 156]
52


IndexError: index 138 is out of bounds for axis 0 with size 52